# Cardiac-Synchronized Learning Task Documentation
Author: Hamed Ghane  
Date: **December 3, 2024**

## Task Overview
A probability learning paradigm synchronized with cardiac cycle timing for feedback presentation.

## Task Structure

### Setup Phase
```
- Operator inputs:
  - Participant ID
  - Session number
  - Run number
  - Number of blocks (n_B)
  - Number of trials per block (n_T)
```

### Block Design
```
- n_B blocks total
- Each block has n_T trials
- Probability reversal at n_T/2
- Trial timing distribution:
  * 1/3 systole
  * 1/3 diastole
  * 1/3 mixed
```

### Probability Structure
```
First Half (n_T/2 trials):
  Symbol A: 70% → reward
  Symbol B: 30% → reward

Second Half (n_T/2 trials):
  Symbol A: 30% → reward
  Symbol B: 70% → reward
```

### Trial Sequence
1. Symbol presentation (A/B, randomized position)
2. Response window (1.25s)
3. R-peak detection
4. Feedback presentation:
   ```
   Duration: 4.0s
   Timing relative to R-peak:
   - Systole: 0-300ms post R-peak
   - Diastole: >300ms post R-peak
   - Mixed: randomly selected from either window
   ```
5. ITI (2-3s)

## Data Stream

### LSL Markers
```
Task Events:
  exp_start/exp_end
  block_start/block_end
  reversal
  trial_start/trial_end
  choice_made
  feedback_onset
  timeout
  instruct_start/instruct_end

Feedback:
  feedback_win
  feedback_loss
  feedback_neutral
```

### Data Output
File: `{participant}-ses{session}-run{run}-{YYYYMMDD-HHMMSS}.csv`

```
Variables per trial:
- Block number
- Trial number
- Response time
- Chosen symbol (A/B)
- Symbol positions
- Feedback type
- Reversal state
- Timing mode
- R-peak time
- Feedback time
- Cardiac phase
- Time from R-peak
```

## Control Keys
```
LEFT ARROW  : Choose left symbol
RIGHT ARROW : Choose right symbol
SPACE      : Progress instructions
Q          : Quit experiment
```

---
*Note: This task requires LSL stream input for R-peak detection*

In [ ]:
from psychopy import visual, core, data, event, gui
import pandas as pd
import numpy as np
from pathlib import Path
from pylsl import StreamInlet, resolve_stream, StreamInfo, StreamOutlet
import threading
import queue
import time

class CardiacSyncedLearningTask:
    def __init__(self):
        # Task settings
        self.settings = {
            'decision_duration': 1.25,  # Decision phase duration
            'delay_min': 1.5,  # Minimum delay duration
            'delay_max': 2.0,  # Maximum delay duration
            'feedback_duration': 4.0,  # Feedback duration
            'iti_min': 2.0,  # Minimum inter-trial interval
            'iti_max': 3.0,  # Maximum inter-trial interval
            'win_probability_good': 0.70,  # Probability of win for good symbol
            'win_probability_bad': 0.30,  # Probability of win for bad symbol
            'systole_duration': 0.300,  # 300ms post R-peak
            'time_resolution': 0.010  # 10ms bins
        }

        # Setup LSL outlet for markers
        self.marker_info = StreamInfo(
            'TaskMarkers',     # Stream name
            'Markers',         # Stream type
            1,                 # Number of channels
            0,                 # Irregular sampling rate
            'string',          # Channel format
            'TaskMarker123'    # Source id
        )
        self.marker_outlet = StreamOutlet(self.marker_info)

        # Marker codes for different events
        self.markers = {
            'experiment_start': 'exp_start',
            'experiment_end': 'exp_end',
            'block_start': 'block_start',
            'block_end': 'block_end',
            'reversal': 'reversal',
            'trial_start': 'trial_start',
            'choice_made': 'choice_made',
            'feedback_onset': 'feedback_onset',
            'trial_end': 'trial_end',
            'instruction_start': 'instruct_start',
            'instruction_end': 'instruct_end',
            'timeout': 'timeout',
            'win_feedback': 'feedback_win',
            'loss_feedback': 'feedback_loss',
            'neutral_feedback': 'feedback_neutral'
        }

        # Setup paths
        self.base_path = Path.cwd()
        self.stim_path = self.base_path / 'stimuli'
        self.data_path = self.base_path / 'data'
        self.data_path.mkdir(exist_ok=True)

        # Define stimuli filenames
        self.stimuli = {
            'symbols': ['A.png', 'B.png', 'C.png'],
            'fixation': 'fix.png',
            'feedback': {
                'win': 'POS.png',
                'loss': 'NEG.png',
                'neutral': 'NEU.png'
            }
        }

        # Task instructions
        self.instructions = [
            """Welcome to the experiment!

In this task, you will see two symbols on each trial.
Your goal is to learn which symbol is more likely to give you rewards.
Note that these probabilities may change during the task.

Press SPACE to continue...""",

            """On each trial:
1. Two symbols will appear
2. Choose the left symbol with the LEFT ARROW key
3. Choose the right symbol with the RIGHT ARROW key
4. You have 1.25 seconds to make your choice

Press SPACE to continue...""",

            """After your choice:
- If you see a WIN image, you earned points
- If you see a LOSS image, you lost points
- If you see a NEUTRAL image, no points change

The same symbols will have different probabilities of winning.
These probabilities will change during each block.

Press SPACE to continue...""",

            """Important Notes:
- Respond as quickly and accurately as possible
- If you don't respond in time, you'll see a neutral outcome
- You can press 'q' at any time to end the experiment
- Your data will be saved automatically

Press SPACE to begin the task."""
        ]

        # LSL and timing variables
        self.r_peak_times = queue.Queue()
        self.current_r_peak = None
        self.next_r_peak = None
        self.cardiac_timing_options = {
            'systole': [],
            'diastole': []
        }

    def send_marker(self, marker_code):
        """Send an LSL marker with the specified code"""
        self.marker_outlet.push_sample([marker_code])
        print(f"LSL Marker sent: {marker_code}")

    def setup_experiment(self):
        """Initialize PsychoPy window and load stimuli"""
        self.win = visual.Window(
            size=[1000, 800],
            fullscr=False,
            units='height',
            color=[0, 0, 0],
            allowGUI=True
        )

        # Load stimuli
        self.stim = {
            'symbols': [
                visual.ImageStim(self.win, image=str(self.stim_path / img))
                for img in self.stimuli['symbols'][:2]  # Only load A and B
            ],
            'fixation': visual.ImageStim(
                self.win,
                image=str(self.stim_path / self.stimuli['fixation'])
            ),
            'feedback': {
                key: visual.ImageStim(self.win, image=str(self.stim_path / path))
                for key, path in self.stimuli['feedback'].items()
            },
            'text': visual.TextStim(
                self.win,
                text='',
                height=0.05,
                wrapWidth=0.8,
                color='white'
            ),
            'timeout_msg': visual.TextStim(
                self.win,
                text='Too Slow, Please Answer Quicker',
                height=0.05,
                wrapWidth=0.8,
                color='red'
            )
        }

    def get_participant_info(self):
        """Show dialog to collect participant information including block and trial numbers"""
        current_time = time.strftime("%Y%m%d-%H%M%S")

        exp_info = {
            'participant': '',
            'session': '001',
            'run': '1',
            'n_blocks': '6',
            'n_trials': '60',
            'date_time': current_time,
        }

        dlg = gui.DlgFromDict(
            dictionary=exp_info,
            title='Task Info',
            fixed=['date_time']
        )

        if dlg.OK:
            # Convert numeric fields to integers
            exp_info['n_blocks'] = int(exp_info['n_blocks'])
            exp_info['n_trials'] = int(exp_info['n_trials'])
            return exp_info
        else:
            core.quit()

    def get_timing_mode(self, trial_num, n_trials):
        """Determine timing mode based on trial position within block"""
        third = n_trials // 3
        if trial_num < third:
            return 'systole'
        elif trial_num < 2 * third:
            return 'diastole'
        else:
            return 'mixed'

    def get_win_probability(self, symbol_index, trial_num, n_trials, is_reversed):
        """Determine win probability based on symbol and trial position"""
        is_symbol_a = symbol_index == 0

        if not is_reversed:  # First half
            if is_symbol_a:
                return self.settings['win_probability_good']
            else:
                return self.settings['win_probability_bad']
        else:  # Second half (after reversal)
            if is_symbol_a:
                return self.settings['win_probability_bad']
            else:
                return self.settings['win_probability_good']

    def setup_lsl(self):
        """Setup LSL inlet for R-peak markers"""
        print("Looking for R-peak markers stream...")
        streams = resolve_stream('type', 'R_PEAK')
        self.inlet = StreamInlet(streams[0])

        # Start R-peak collection thread
        self.lsl_thread = threading.Thread(target=self.collect_r_peaks)
        self.lsl_thread.daemon = True
        self.lsl_thread.start()

    def collect_r_peaks(self):
        """Continuously collect R-peak markers"""
        while True:
            sample, timestamp = self.inlet.pull_sample()
            self.r_peak_times.put(timestamp)

    def show_instructions(self):
        """Display task instructions with markers"""
        self.send_marker(self.markers['instruction_start'])

        for instruction in self.instructions:
            self.stim['text'].text = instruction
            self.stim['text'].draw()
            self.win.flip()

            while True:
                keys = event.waitKeys(keyList=['space', 'q'])
                if 'q' in keys:
                    self.win.close()
                    core.quit()
                if 'space' in keys:
                    break

        self.send_marker(self.markers['instruction_end'])

    def get_cardiac_timing_options(self, r_peak_time, next_r_peak_time):
        """Generate timing options relative to R-peak"""
        options = {'systole': [], 'diastole': []}
        current_time = r_peak_time

        while current_time < next_r_peak_time:
            delay = current_time - r_peak_time
            if delay <= self.settings['systole_duration']:
                options['systole'].append(current_time)
            else:
                options['diastole'].append(current_time)
            current_time += self.settings['time_resolution']

        return options

    def select_feedback_time(self, timing_mode):
        """Select feedback presentation time based on mode"""
        options = []
        if timing_mode == 'systole':
            options = self.cardiac_timing_options['systole']
        elif timing_mode == 'diastole':
            options = self.cardiac_timing_options['diastole']
        else:  # mixed
            options = (self.cardiac_timing_options['systole'] +
                      self.cardiac_timing_options['diastole'])

        return np.random.choice(options) if options else time.time()

    def run_trial(self, trial_num, n_trials, is_reversed):
        """Run a single trial with cardiac-synced feedback and markers"""
        # Determine timing mode based on trial position
        timing_mode = self.get_timing_mode(trial_num, n_trials)

        # Send trial start marker
        self.send_marker(self.markers['trial_start'])

        # Check for quit key
        if 'q' in event.getKeys(keyList=['q']):
            self.win.close()
            core.quit()

        # Decision Phase
        positions = [(-0.15, 0), (0.15, 0)]
        symbol_indices = [0, 1]  # 0 for A, 1 for B

        # Randomize positions while keeping track of which symbol is which
        combined = list(zip(self.stim['symbols'], positions, symbol_indices))
        np.random.shuffle(combined)
        symbols, positions, indices = zip(*combined)

        for sym, pos in zip(symbols, positions):
            sym.pos = pos
            sym.draw()
        self.win.flip()

        # Get response with timeout
        timer = core.Clock()
        keys = event.waitKeys(
            maxWait=self.settings['decision_duration'],
            keyList=['left', 'right', 'q'],
            timeStamped=timer
        )

        if keys and 'q' in keys[0]:
            self.win.close()
            core.quit()

        # Process response and prepare feedback
        if not keys:  # No response
            self.send_marker(self.markers['timeout'])
            self.stim['timeout_msg'].draw()
            self.win.flip()
            core.wait(1.0)

            feedback_type = 'neutral'
            rt = None
            choice = None
            correct = False
            chosen_symbol = None
        else:
            self.send_marker(self.markers['choice_made'])
            key, rt = keys[0]
            choice = 0 if key == 'left' else 1
            chosen_symbol = indices[choice]

            # Get win probability for chosen symbol
            win_prob = self.get_win_probability(
                chosen_symbol, trial_num, n_trials, is_reversed
            )

            outcome = np.random.random() < win_prob
            feedback_type = 'win' if outcome else 'loss'

        # Wait for next R-peak and prepare feedback timing
        r_peak = self.r_peak_times.get()
        next_r_peak = self.r_peak_times.get()

        # Generate timing options for this cardiac cycle
        self.cardiac_timing_options = self.get_cardiac_timing_options(
            r_peak, next_r_peak
        )

        # Select feedback time
        feedback_time = self.select_feedback_time(timing_mode)

        # Wait until selected feedback time
        while time.time() < feedback_time:
            core.wait(0.001)

        # Show feedback with appropriate marker
        feedback_marker = {
            'win': self.markers['win_feedback'],
            'loss': self.markers['loss_feedback'],
            'neutral': self.markers['neutral_feedback']
        }[feedback_type]

        self.send_marker(self.markers['feedback_onset'])
        self.send_marker(feedback_marker)

        self.stim['feedback'][feedback_type].draw()
        self.win.flip()
        core.wait(self.settings['feedback_duration'])

        # Send trial end marker
        self.send_marker(self.markers['trial_end'])

        # Create trial data dictionary
        trial_data = {
            'rt': rt,
            'choice': choice,
            'chosen_symbol': 'A' if chosen_symbol == 0 else 'B',
            'symbol_positions': 'AB' if indices[0] == 0 else 'BA',
            'feedback': feedback_type,
            'is_reversed': is_reversed,
            'timing_mode': timing_mode,
            'r_peak_time': r_peak,
            'next_r_peak_time': next_r_peak,
            'feedback_time': feedback_time,
            'cardiac_phase': ('systole' if feedback_time - r_peak <=
                            self.settings['systole_duration'] else 'diastole'),
            'time_from_r_peak': feedback_time - r_peak
        }

        # Print trial report
        self.print_trial_report(trial_data, trial_num, is_reversed)

        # Inter-trial interval
        self.stim['fixation'].draw()
        self.win.flip()
        core.wait(np.random.uniform(
            self.settings['iti_min'],
            self.settings['iti_max']
        ))

        return trial_data

    def print_trial_report(self, trial_data, trial_num, is_reversed):
        """Print a detailed report of the trial results"""
        print("\n" + "="*50)
        print(f"Trial {trial_num} Report (Reversed: {is_reversed})")
        print("-"*50)

        if trial_data['choice'] is not None:
            print(f"Chosen Symbol: {trial_data['chosen_symbol']}")
            print(f"Symbol Positions: {trial_data['symbol_positions']}")
            print(f"Response Time: {trial_data['rt']:.3f}s")
        else:
            print("Response: No response (Too slow)")

        print(f"Feedback: {trial_data['feedback'].upper()}")
        print(f"Timing Mode: {trial_data['timing_mode']}")
        print(f"Cardiac Phase: {trial_data['cardiac_phase']}")
        print(f"Time from R-peak: {trial_data['time_from_r_peak']*1000:.1f}ms")
        print("="*50 + "\n")

    def run_experiment(self):
        """Run the complete experiment with markers"""
        # Setup
        participant_info = self.get_participant_info()
        self.setup_experiment()

        # Send experiment start marker
        self.send_marker(self.markers['experiment_start'])

        # Setup LSL
        self.setup_lsl()
        print("Calibrating R-peak detection...")
        core.wait(3.0)

        # Show instructions
        self.show_instructions()

        # Create data storage
        all_data = []

        try:
            # Run blocks
            for block in range(participant_info['n_blocks']):
                # Send block start marker
                self.send_marker(self.markers['block_start'])
                print(f"\nStarting Block {block + 1}...")

                n_trials = participant_info['n_trials']
                reversal_point = n_trials // 2

                for trial in range(n_trials):
                    # Check if we've reached reversal point
                    is_reversed = trial >= reversal_point
                    if trial == reversal_point:
                        self.send_marker(self.markers['reversal'])
                        print("\nProbability Reversal!")

                    trial_data = self.run_trial(trial, n_trials, is_reversed)
                    trial_data.update({
                        'block': block,
                        'trial': trial,
                        'participant': participant_info['participant'],
                        'session': participant_info['session'],
                        'run': participant_info['run'],
                        'date_time': participant_info['date_time']
                    })
                    all_data.append(trial_data)

                # Send block end marker
                self.send_marker(self.markers['block_end'])
                print(f"\nBlock {block + 1} completed.")

        finally:
            # Send experiment end marker
            self.send_marker(self.markers['experiment_end'])

            if all_data:
                df = pd.DataFrame(all_data)
                filename = f"{participant_info['participant']}-ses{participant_info['session']}-run{participant_info['run']}-{participant_info['date_time']}.csv"
                df.to_csv(self.data_path / filename, index=False)
                print(f"\nData saved as: {filename}")

            self.win.close()

if __name__ == "__main__":
    task = CardiacSyncedLearningTask()
    task.run_experiment()



# Thers is a Major Issue
In the original version, there was a logical impossibility in the diastole timing:

- The script waited for the second R-peak to define the complete timing grid.
- But it needed to present feedback in the diastole phase before that second R-peak occurred.
- This created a paradox: we couldn't wait for information from the future (second R-peak) to make decisions about timing between the first and second R-peak.

## Conservative Solution
The current version solves this by:

- Always waiting for **TWO R-peaks after the choice before presenting any feedback**.
- Using the second R-peak as the reference point for both systole and diastole timing:

  - **Systole**: 0-300ms after second R-peak
  - **Diastole**: 300-700ms after second R-peak

This ensures all timing information is available before making presentation decisions.

## Key Code Section

```python
# Wait for TWO R-peaks after choice for conservative timing
r_peak_1 = self.r_peak_times.get()  # First R-peak after choice
r_peak_2 = self.r_peak_times.get()  # Second R-peak

# Generate timing options for second R-peak
timing_options = self.get_timing_options(r_peak_2)

# Select feedback time
feedback_time = self.select_feedback_time(timing_mode, timing_options)


In [ ]:
from psychopy import visual, core, data, event, gui
import pandas as pd
import numpy as np
from pathlib import Path
from pylsl import StreamInlet, resolve_stream, StreamInfo, StreamOutlet
import threading
import queue
import time

class CardiacSyncedLearningTask:
    def __init__(self):
        # Task settings
        self.settings = {
            'decision_duration': 1.25,  # Decision phase duration
            'delay_min': 1.5,  # Minimum delay duration
            'delay_max': 2.0,  # Maximum delay duration
            'feedback_duration': 4.0,  # Feedback duration
            'iti_min': 2.0,  # Minimum inter-trial interval
            'iti_max': 3.0,  # Maximum inter-trial interval
            'win_probability_good': 0.70,  # Probability of win for good symbol
            'win_probability_bad': 0.30,  # Probability of win for bad symbol
            'systole_window_start': 0,      # ms after R-peak
            'systole_window_end': 300,      # ms after R-peak
            'diastole_window_start': 300,   # ms after R-peak
            'diastole_window_end': 700,     # ms after R-peak
            'grid_resolution': 10           # ms steps
        }

        # Setup LSL outlet for markers
        self.marker_info = StreamInfo(
            'TaskMarkers',     # Stream name
            'Markers',         # Stream type
            1,                 # Number of channels
            0,                 # Irregular sampling rate
            'string',          # Channel format
            'TaskMarker123'    # Source id
        )
        self.marker_outlet = StreamOutlet(self.marker_info)

        # Marker codes
        self.markers = {
            'experiment_start': 'exp_start',
            'experiment_end': 'exp_end',
            'block_start': 'block_start',
            'block_end': 'block_end',
            'reversal': 'reversal',
            'trial_start': 'trial_start',
            'choice_made': 'choice_made',
            'feedback_onset': 'feedback_onset',
            'trial_end': 'trial_end',
            'instruction_start': 'instruct_start',
            'instruction_end': 'instruct_end',
            'timeout': 'timeout',
            'win_feedback': 'feedback_win',
            'loss_feedback': 'feedback_loss',
            'neutral_feedback': 'feedback_neutral'
        }

        # Setup paths
        self.base_path = Path.cwd()
        self.stim_path = self.base_path / 'stimuli'
        self.data_path = self.base_path / 'data'
        self.data_path.mkdir(exist_ok=True)

        # Define stimuli filenames
        self.stimuli = {
            'symbols': ['A.png', 'B.png', 'C.png'],
            'fixation': 'fix.png',
            'feedback': {
                'win': 'POS.png',
                'loss': 'NEG.png',
                'neutral': 'NEU.png'
            }
        }

        # Task instructions
        self.instructions = [
            """Welcome to the experiment!

In this task, you will see two symbols on each trial.
Your goal is to learn which symbol is more likely to give you rewards.
Note that these probabilities may change during the task.

Press SPACE to continue...""",

            """On each trial:
1. Two symbols will appear
2. Choose the left symbol with the LEFT ARROW key
3. Choose the right symbol with the RIGHT ARROW key
4. You have 1.25 seconds to make your choice

Press SPACE to continue...""",

            """After your choice:
- If you see a WIN image, you earned points
- If you see a LOSS image, you lost points
- If you see a NEUTRAL image, no points change

The same symbols will have different probabilities of winning.
These probabilities will change during each block.

Press SPACE to continue...""",

            """Important Notes:
- Respond as quickly and accurately as possible
- If you don't respond in time, you'll see a neutral outcome
- You can press 'q' at any time to end the experiment
- Your data will be saved automatically

Press SPACE to begin the task."""
        ]

        # Initialize queue for R-peak times
        self.r_peak_times = queue.Queue()

    def send_marker(self, marker_code):
        """Send an LSL marker with the specified code"""
        self.marker_outlet.push_sample([marker_code])
        print(f"LSL Marker sent: {marker_code}")

    def setup_experiment(self):
        """Initialize PsychoPy window and load stimuli"""
        self.win = visual.Window(
            size=[1000, 800],
            fullscr=False,
            units='height',
            color=[0, 0, 0],
            allowGUI=True
        )

        # Load stimuli
        self.stim = {
            'symbols': [
                visual.ImageStim(self.win, image=str(self.stim_path / img))
                for img in self.stimuli['symbols'][:2]  # Only load A and B
            ],
            'fixation': visual.ImageStim(
                self.win,
                image=str(self.stim_path / self.stimuli['fixation'])
            ),
            'feedback': {
                key: visual.ImageStim(self.win, image=str(self.stim_path / path))
                for key, path in self.stimuli['feedback'].items()
            },
            'text': visual.TextStim(
                self.win,
                text='',
                height=0.05,
                wrapWidth=0.8,
                color='white'
            ),
            'timeout_msg': visual.TextStim(
                self.win,
                text='Too Slow, Please Answer Quicker',
                height=0.05,
                wrapWidth=0.8,
                color='red'
            )
        }

    def setup_lsl(self):
        """Setup LSL inlet for R-peak markers"""
        print("Looking for R-peak markers stream...")
        streams = resolve_stream('type', 'R_PEAK')
        self.inlet = StreamInlet(streams[0])

        # Start R-peak collection thread
        self.lsl_thread = threading.Thread(target=self.collect_r_peaks)
        self.lsl_thread.daemon = True
        self.lsl_thread.start()

    def collect_r_peaks(self):
        """Continuously collect R-peak markers"""
        while True:
            sample, timestamp = self.inlet.pull_sample()
            self.r_peak_times.put(timestamp)

    def get_participant_info(self):
        """Show dialog to collect participant information"""
        current_time = time.strftime("%Y%m%d-%H%M%S")

        exp_info = {
            'participant': '',
            'session': '001',
            'run': '1',
            'n_blocks': '6',
            'n_trials': '60',
            'date_time': current_time,
        }

        dlg = gui.DlgFromDict(
            dictionary=exp_info,
            title='Task Info',
            fixed=['date_time']
        )

        if dlg.OK:
            # Convert numeric fields to integers
            exp_info['n_blocks'] = int(exp_info['n_blocks'])
            exp_info['n_trials'] = int(exp_info['n_trials'])
            return exp_info
        else:
            core.quit()

    def show_instructions(self):
        """Display task instructions with markers"""
        self.send_marker(self.markers['instruction_start'])

        for instruction in self.instructions:
            self.stim['text'].text = instruction
            self.stim['text'].draw()
            self.win.flip()

            while True:
                keys = event.waitKeys(keyList=['space', 'q'])
                if 'q' in keys:
                    self.win.close()
                    core.quit()
                if 'space' in keys:
                    break

        self.send_marker(self.markers['instruction_end'])

    def get_timing_mode(self, trial_num, n_trials):
        """Determine timing mode based on trial position within block"""
        third = n_trials // 3
        if trial_num < third:
            return 'systole'
        elif trial_num < 2 * third:
            return 'diastole'
        else:
            return 'mixed'

    def get_win_probability(self, symbol_index, trial_num, n_trials, is_reversed):
        """Determine win probability based on symbol and trial position"""
        is_symbol_a = symbol_index == 0

        if not is_reversed:  # First half
            if is_symbol_a:
                return self.settings['win_probability_good']
            else:
                return self.settings['win_probability_bad']
        else:  # Second half (after reversal)
            if is_symbol_a:
                return self.settings['win_probability_bad']
            else:
                return self.settings['win_probability_good']

    def get_timing_options(self, r_peak_time):
        """Generate timing options relative to R-peak with fixed windows"""
        options = {'systole': [], 'diastole': []}

        # Create systole grid
        for t in range(self.settings['systole_window_start'],
                      self.settings['systole_window_end'],
                      self.settings['grid_resolution']):
            options['systole'].append(r_peak_time + t/1000)

        # Create diastole grid
        for t in range(self.settings['diastole_window_start'],
                      self.settings['diastole_window_end'],
                      self.settings['grid_resolution']):
            options['diastole'].append(r_peak_time + t/1000)

        return options

    def select_feedback_time(self, timing_mode, timing_options):
        """Select feedback presentation time based on mode"""
        if timing_mode == 'systole':
            options = timing_options['systole']
        elif timing_mode == 'diastole':
            options = timing_options['diastole']
        else:  # mixed
            options = timing_options['systole'] + timing_options['diastole']

        return np.random.choice(options) if options else time.time()

    def run_trial(self, trial_num, n_trials, is_reversed):
        """Run a single trial with conservative cardiac-synced feedback"""
        # Determine timing mode
        timing_mode = self.get_timing_mode(trial_num, n_trials)

        # Send trial start marker
        self.send_marker(self.markers['trial_start'])

        # Decision Phase
        t_0 = time.time()  # Response time

        # Present symbols and get response
        positions = [(-0.15, 0), (0.15, 0)]
        symbol_indices = [0, 1]

        # Randomize positions
        combined = list(zip(self.stim['symbols'], positions, symbol_indices))
        np.random.shuffle(combined)
        symbols, positions, indices = zip(*combined)

        for sym, pos in zip(symbols, positions):
            sym.pos = pos
            sym.draw()
        self.win.flip()

        # Get response with timeout
        timer = core.Clock()
        keys = event.waitKeys(
            maxWait=self.settings['decision_duration'],
            keyList=['left', 'right', 'q'],
            timeStamped=timer
        )

        if keys and 'q' in keys[0]:
            self.win.close()
            core.quit()

        # Process response
        if not keys:
            self.send_marker(self.markers['timeout'])
            self.stim['timeout_msg'].draw()
            self.win.flip()
            core.wait(1.0)
            feedback_type = 'neutral'
            rt = None
            choice = None
            chosen_symbol = None
        else:
            self.send_marker(self.markers['choice_made'])
            key, rt = keys[0]
            choice = 0 if key == 'left' else 1
            chosen_symbol = indices[choice]

            # Determine outcome
            win_prob = self.get_win_probability(
                chosen_symbol, trial_num, n_trials, is_reversed
            )
            outcome = np.random.random() < win_prob
            feedback_type = 'win' if outcome else 'loss'

        # Wait for TWO R-peaks after choice for conservative timing
        r_peak_1 = self.r_peak_times.get()  # First R-peak after choice
        r_peak_2 = self.r_peak_times.get()  # Second R-peak

        # Generate timing options for second R-peak
        timing_options = self.get_timing_options(r_peak_2)

        # Select feedback time
        feedback_time = self.select_feedback_time(timing_mode, timing_options)

        # Wait until selected feedback time
        while time.time() < feedback_time:
            core.wait(0.001)

        # Show feedback
        feedback_marker = {
            'win': self.markers['win_feedback'],
            'loss': self.markers['loss_feedback'],
            'neutral': self.markers['neutral_feedback']
        }[feedback_type]

        self.send_marker(self.markers['feedback_onset'])
        self.send_marker(feedback_marker)

        self.stim['feedback'][feedback_type].draw()
        self.win.flip()
        core.wait(self.settings['feedback_duration'])

        # Send trial end marker
        self.send_marker(self.markers['trial_end'])

        # Calculate timing information
        cardiac_phase = ('systole' if feedback_time - r_peak_2 <= 0.300 else 'diastole')

        # Create trial data
        trial_data = {
            'rt': rt,
            'choice': choice,
            'chosen_symbol': 'A' if chosen_symbol == 0 else 'B',
            'symbol_positions': 'AB' if indices[0] == 0 else 'BA',
            'feedback': feedback_type,
            'is_reversed': is_reversed,
            'timing_mode': timing_mode,
            'r_peak_time': r_peak_2,  # Using second R-peak as reference
            'feedback_time': feedback_time,
            'cardiac_phase': cardiac_phase,
            'time_from_r_peak': feedback_time - r_peak_2,
            'response_time': t_0,
            'first_r_peak': r_peak_1,
            'second_r_peak': r_peak_2
        }

        # Print trial report
        self.print_trial_report(trial_data, trial_num, is_reversed)

        # Inter-trial interval
        self.stim['fixation'].draw()
        self.win.flip()
        core.wait(np.random.uniform(
            self.settings['iti_min'],
            self.settings['iti_max']
        ))

        return trial_data

    def print_trial_report(self, trial_data, trial_num, is_reversed):
        """Print a detailed report of the trial results"""
        print("\n" + "="*50)
        print(f"Trial {trial_num} Report (Reversed: {is_reversed})")
        print("-"*50)

        if trial_data['choice'] is not None:
            print(f"Chosen Symbol: {trial_data['chosen_symbol']}")
            print(f"Symbol Positions: {trial_data['symbol_positions']}")
            print(f"Response Time: {trial_data['rt']:.3f}s")
        else:
            print("Response: No response (Too slow)")

        print(f"Feedback: {trial_data['feedback'].upper()}")
        print(f"Timing Mode: {trial_data['timing_mode']}")
        print(f"Cardiac Phase: {trial_data['cardiac_phase']}")
        print(f"Time from R-peak: {trial_data['time_from_r_peak']*1000:.1f}ms")
        print("="*50 + "\n")

    def run_experiment(self):
        """Run the complete experiment with markers"""
        # Setup
        participant_info = self.get_participant_info()
        self.setup_experiment()

        # Send experiment start marker
        self.send_marker(self.markers['experiment_start'])

        # Setup LSL
        self.setup_lsl()
        print("Waiting for R-peak stream...")
        core.wait(3.0)

        # Show instructions
        self.show_instructions()

        # Create data storage
        all_data = []

        try:
            # Run blocks
            for block in range(participant_info['n_blocks']):
                # Send block start marker
                self.send_marker(self.markers['block_start'])
                print(f"\nStarting Block {block + 1}...")

                n_trials = participant_info['n_trials']
                reversal_point = n_trials // 2

                for trial in range(n_trials):
                    # Check if we've reached reversal point
                    is_reversed = trial >= reversal_point
                    if trial == reversal_point:
                        self.send_marker(self.markers['reversal'])
                        print("\nProbability Reversal!")

                    trial_data = self.run_trial(trial, n_trials, is_reversed)
                    trial_data.update({
                        'block': block,
                        'trial': trial,
                        'participant': participant_info['participant'],
                        'session': participant_info['session'],
                        'run': participant_info['run'],
                        'date_time': participant_info['date_time']
                    })
                    all_data.append(trial_data)

                # Send block end marker
                self.send_marker(self.markers['block_end'])
                print(f"\nBlock {block + 1} completed.")

        finally:
            # Send experiment end marker
            self.send_marker(self.markers['experiment_end'])

            if all_data:
                df = pd.DataFrame(all_data)
                filename = f"{participant_info['participant']}-ses{participant_info['session']}-run{participant_info['run']}-{participant_info['date_time']}.csv"
                df.to_csv(self.data_path / filename, index=False)
                print(f"\nData saved as: {filename}")

            self.win.close()

if __name__ == "__main__":
    task = CardiacSyncedLearningTask()
    task.run_experiment()



# Adaptive Solution

## Overview
Instead of waiting for two R-peaks to determine diastole timing, the adaptive solution uses real-time heart rate information to predict the next R-peak timing.

## Key Components

### CardiacCycleTracker Class
1. Maintains a rolling window of recent R-R intervals
2. Continuously updates average heart rate
3. Provides predicted timing for next R-peak

```python
class CardiacCycleTracker:
   def __init__(self, window_size=30):
       self.window_size = window_size
       self.r_r_intervals = []
       self.mean_rr = None
```

### Adaptive Timing Process

- **During instructions/calibration phase:**
  - Collect 30 seconds of R-peaks
  - Calculate baseline R-R interval average

- **During trials:**
  - Wait for first R-peak after choice
  - Calculate timing grids based on average R-R:
    - Systole: 0-300ms after R-peak
    - Diastole: 300ms to average R-R time

### Key Adaptive Code

```python
def get_timing_options(self, r_peak_time):
    average_r_r = self.cardiac_tracker.get_average_r_r()
    options = {'systole': [], 'diastole': []}
    
    # Systole: fixed 0-300ms
    for t in range(0, 300, 10):  # 10ms steps
        options['systole'].append(r_peak_time + t/1000)
    
    # Diastole: 300ms to predicted next R-peak
    for t in range(300, int(average_r_r*1000), 10):
        options['diastole'].append(r_peak_time + t/1000)
```

## Advantages
- More immediate feedback presentation
- Adapts to individual heart rate variations
- Accounts for heart rate changes during task

## Trade-offs
- Relies on heart rate stability
- May occasionally miss timing if heart rate suddenly changes
- Requires initial calibration period

This adaptive approach provides more natural timing but requires careful validation of heart rate predictions, while the conservative approach sacrifices immediacy for timing certainty.
```

In [ ]:
from psychopy import visual, core, data, event, gui
import pandas as pd
import numpy as np
from pathlib import Path
from pylsl import StreamInlet, resolve_stream, StreamInfo, StreamOutlet
import threading
import queue
import time

class CardiacCycleTracker:
    def __init__(self, window_size=30):
        self.window_size = window_size
        self.r_peaks = []
        self.r_r_intervals = []
        self.lock = threading.Lock()
        self.mean_rr = None

    def update(self, r_peak_time):
        with self.lock:
            if self.r_peaks:
                interval = r_peak_time - self.r_peaks[-1]
                if 0.4 <= interval <= 1.5:  # Basic validity check for R-R interval
                    self.r_r_intervals.append(interval)
                    if len(self.r_r_intervals) > self.window_size:
                        self.r_r_intervals.pop(0)
                    self.mean_rr = np.mean(self.r_r_intervals)

            self.r_peaks.append(r_peak_time)
            if len(self.r_peaks) > self.window_size:
                self.r_peaks.pop(0)

    def get_average_r_r(self):
        with self.lock:
            if self.mean_rr is None or len(self.r_r_intervals) < 5:
                return 1.0  # Default 1 second if not enough data
            return self.mean_rr

class CardiacSyncedLearningTask:
    def __init__(self):
        # Initialize experiment start time and cardiac tracker
        self.experiment_start_time = None
        self.cardiac_tracker = CardiacCycleTracker()

        # Task settings
        self.settings = {
            'decision_duration': 1.25,  # Decision phase duration
            'delay_min': 1.5,  # Minimum delay duration
            'delay_max': 2.0,  # Maximum delay duration
            'feedback_duration': 4.0,  # Feedback duration
            'iti_min': 2.0,  # Minimum inter-trial interval
            'iti_max': 3.0,  # Maximum inter-trial interval
            'win_probability_good': 0.70,  # Probability of win for good symbol
            'win_probability_bad': 0.30,  # Probability of win for bad symbol
            'systole_duration': 0.300,  # 300ms post R-peak
            'grid_resolution': 0.010,  # 10ms steps
            'calibration_duration': 30.0  # 30 seconds calibration
        }

        # Setup LSL outlet for markers
        self.marker_info = StreamInfo(
            'TaskMarkers',     # Stream name
            'Markers',         # Stream type
            1,                 # Number of channels
            0,                 # Irregular sampling rate
            'string',          # Channel format
            'TaskMarker123'    # Source id
        )
        self.marker_outlet = StreamOutlet(self.marker_info)

        # Marker codes
        self.markers = {
            'experiment_start': 'exp_start',
            'experiment_end': 'exp_end',
            'block_start': 'block_start',
            'block_end': 'block_end',
            'reversal': 'reversal',
            'trial_start': 'trial_start',
            'choice_made': 'choice_made',
            'feedback_onset': 'feedback_onset',
            'trial_end': 'trial_end',
            'instruction_start': 'instruct_start',
            'instruction_end': 'instruct_end',
            'timeout': 'timeout',
            'win_feedback': 'feedback_win',
            'loss_feedback': 'feedback_loss',
            'neutral_feedback': 'feedback_neutral',
            'timing_warning': 'timing_warning',
            'calibration_start': 'calib_start',
            'calibration_end': 'calib_end'
        }

        # Setup paths
        self.base_path = Path.cwd()
        self.stim_path = self.base_path / 'stimuli'
        self.data_path = self.base_path / 'data'
        self.data_path.mkdir(exist_ok=True)

        # Define stimuli filenames
        self.stimuli = {
            'symbols': ['A.png', 'B.png', 'C.png'],
            'fixation': 'fix.png',
            'feedback': {
                'win': 'POS.png',
                'loss': 'NEG.png',
                'neutral': 'NEU.png'
            }
        }

        # Add calibration instruction
        calibration_instruction = """
        We will now calibrate the system to your heart rate.
        Please sit still and relax for the next 30 seconds.

        Press SPACE to begin the calibration..."""

        # Task instructions
        self.instructions = [
            """Welcome to the experiment!

In this task, you will see two symbols on each trial.
Your goal is to learn which symbol is more likely to give you rewards.
Note that these probabilities may change during the task.

Press SPACE to continue...""",
            calibration_instruction,
            """On each trial:
1. Two symbols will appear
2. Choose the left symbol with the LEFT ARROW key
3. Choose the right symbol with the RIGHT ARROW key
4. You have 1.25 seconds to make your choice

Press SPACE to continue...""",
            """After your choice:
- If you see a WIN image, you earned points
- If you see a LOSS image, you lost points
- If you see a NEUTRAL image, no points change

The same symbols will have different probabilities of winning.
These probabilities will change during each block.

Press SPACE to continue...""",
            """Important Notes:
- Respond as quickly and accurately as possible
- If you don't respond in time, you'll see a neutral outcome
- You can press 'q' at any time to end the experiment
- Your data will be saved automatically

Press SPACE to begin the task."""
        ]

        # Initialize queue for R-peak times
        self.r_peak_times = queue.Queue()

        # Initialize timing log
        self.timing_log = []

    def get_participant_info(self):
        """Show dialog to collect participant information"""
        current_time = time.strftime("%Y%m%d-%H%M%S")

        exp_info = {
            'participant': '',
            'session': '001',
            'run': '1',
            'n_blocks': '6',
            'n_trials': '60',
            'date_time': current_time,
        }

        dlg = gui.DlgFromDict(
            dictionary=exp_info,
            title='Task Info',
            fixed=['date_time']
        )

        if dlg.OK:
            # Convert numeric fields to integers
            exp_info['n_blocks'] = int(exp_info['n_blocks'])
            exp_info['n_trials'] = int(exp_info['n_trials'])
            return exp_info
        else:
            core.quit()

    def send_marker(self, marker_code):
        """Send an LSL marker with the specified code"""
        self.marker_outlet.push_sample([marker_code])
        print(f"LSL Marker sent: {marker_code}")

    def setup_experiment(self):
        """Initialize PsychoPy window and load stimuli"""
        self.win = visual.Window(
            size=[1000, 800],
            fullscr=False,
            units='height',
            color=[0, 0, 0],
            allowGUI=True
        )

        # Load stimuli
        self.stim = {
            'symbols': [
                visual.ImageStim(self.win, image=str(self.stim_path / img))
                for img in self.stimuli['symbols'][:2]  # Only load A and B
            ],
            'fixation': visual.ImageStim(
                self.win,
                image=str(self.stim_path / self.stimuli['fixation'])
            ),
            'feedback': {
                key: visual.ImageStim(self.win, image=str(self.stim_path / path))
                for key, path in self.stimuli['feedback'].items()
            },
            'text': visual.TextStim(
                self.win,
                text='',
                height=0.05,
                wrapWidth=0.8,
                color='white'
            ),
            'timeout_msg': visual.TextStim(
                self.win,
                text='Too Slow, Please Answer Quicker',
                height=0.05,
                wrapWidth=0.8,
                color='red'
            ),
            'calibration_msg': visual.TextStim(
                self.win,
                text='Calibrating...',
                height=0.05,
                wrapWidth=0.8,
                color='white'
            )
        }

    def setup_lsl(self):
        """Setup LSL inlet for R-peak markers"""
        print("Looking for R-peak markers stream...")
        streams = resolve_stream('type', 'R_PEAK')
        self.inlet = StreamInlet(streams[0])

        # Start R-peak collection thread
        self.lsl_thread = threading.Thread(target=self.collect_r_peaks)
        self.lsl_thread.daemon = True
        self.lsl_thread.start()

    def collect_r_peaks(self):
        """Continuously collect R-peak markers and update cardiac tracker"""
        while True:
            sample, timestamp = self.inlet.pull_sample()
            self.r_peak_times.put(timestamp)
            self.cardiac_tracker.update(timestamp)

    def run_calibration(self):
        """Run calibration period to establish baseline heart rate"""
        self.send_marker(self.markers['calibration_start'])
        self.stim['calibration_msg'].draw()
        self.win.flip()

        calibration_start = time.time()
        while time.time() - calibration_start < self.settings['calibration_duration']:
            if 'q' in event.getKeys():
                self.win.close()
                core.quit()
            self.stim['fixation'].draw()
            self.win.flip()

        self.send_marker(self.markers['calibration_end'])
        avg_rr = self.cardiac_tracker.get_average_r_r()
        print(f"Calibration complete. Average R-R interval: {avg_rr:.3f}s")

    def validate_timing(self, feedback_time, r_peak, timing_mode):
        """Validate feedback timing relative to R-peak"""
        time_from_peak = feedback_time - r_peak
        valid = True
        warning_msg = None

        if timing_mode == 'systole':
            if not (0 <= time_from_peak <= self.settings['systole_duration']):
                valid = False
                warning_msg = f"Warning: Systole timing outside window: {time_from_peak:.3f}s"
        else:  # diastole or mixed
            r_r = self.cardiac_tracker.get_average_r_r()
            if not (self.settings['systole_duration'] <= time_from_peak <= r_r):
                valid = False
                warning_msg = f"Warning: Diastole timing outside window: {time_from_peak:.3f}s"

        if not valid:
            print(warning_msg)
            self.send_marker(self.markers['timing_warning'])
            self.timing_log.append({
                'time': time.time() - self.experiment_start_time,
                'warning': warning_msg
            })

        return valid

    def get_timing_options(self, r_peak_time):
        """Generate timing options using adaptive R-R interval"""
        average_r_r = self.cardiac_tracker.get_average_r_r()
        options = {'systole': [], 'diastole': []}

        # Systole grid (0-300ms)
        current_time = r_peak_time
        while current_time <= r_peak_time + self.settings['systole_duration']:
            options['systole'].append(current_time)
            current_time += self.settings['grid_resolution']

        # Diastole grid (300ms-average_R_R)
        current_time = r_peak_time + self.settings['systole_duration']
        while current_time < r_peak_time + average_r_r:
            options['diastole'].append(current_time)
            current_time += self.settings['grid_resolution']

        return options

    def select_feedback_time(self, timing_mode, timing_options):
        """Select feedback presentation time based on mode"""
        if timing_mode == 'systole':
            options = timing_options['systole']
        elif timing_mode == 'diastole':
            options = timing_options['diastole']
        else:  # mixed
            options = timing_options['systole'] + timing_options['diastole']

        return np.random.choice(options) if options else time.time()

    def get_timing_mode(self, trial_num, n_trials):
        """Determine timing mode based on trial position within block"""
        third = n_trials // 3
        if trial_num < third:
            return 'systole'
        elif trial_num < 2 * third:
            return 'diastole'
        else:
            return 'mixed'

    def get_win_probability(self, symbol_index, trial_num, n_trials, is_reversed):
        """Determine win probability based on symbol and trial position"""
        is_symbol_a = symbol_index == 0

        if not is_reversed:  # First half
            if is_symbol_a:
                return self.settings['win_probability_good']
            else:
                return self.settings['win_probability_bad']
        else:  # Second half (after reversal)
            if is_symbol_a:
                return self.settings['win_probability_bad']
            else:
                return self.settings['win_probability_good']
    def print_trial_report(self, trial_data, trial_num, is_reversed):
        """Print a detailed report of the trial results"""
        print("\n" + "="*50)
        print(f"Trial {trial_num} Report (Reversed: {is_reversed})")
        print("-"*50)

        if trial_data['choice'] is not None:
            print(f"Chosen Symbol: {trial_data['chosen_symbol']}")
            print(f"Symbol Positions: {trial_data['symbol_positions']}")
            print(f"Response Time: {trial_data['rt']:.3f}s")
        else:
            print("Response: No response (Too slow)")

        print(f"Feedback: {trial_data['feedback'].upper()}")
        print(f"Timing Mode: {trial_data['timing_mode']}")
        print(f"Cardiac Phase: {trial_data['cardiac_phase']}")
        print(f"Time from R-peak: {trial_data['time_from_r_peak']*1000:.1f}ms")
        print(f"Expected R-R: {trial_data['expected_r_r']*1000:.1f}ms")
        print("="*50 + "\n")

    def run_trial(self, trial_num, n_trials, is_reversed):
        """Run a single trial with adaptive cardiac-synced feedback"""
        # Determine timing mode
        timing_mode = self.get_timing_mode(trial_num, n_trials)

        # Send trial start marker
        self.send_marker(self.markers['trial_start'])

        # Decision Phase
        choice_time = time.time()

        # Present symbols and get response
        positions = [(-0.15, 0), (0.15, 0)]
        symbol_indices = [0, 1]

        # Randomize positions
        combined = list(zip(self.stim['symbols'], positions, symbol_indices))
        np.random.shuffle(combined)
        symbols, positions, indices = zip(*combined)

        for sym, pos in zip(symbols, positions):
            sym.pos = pos
            sym.draw()
        self.win.flip()

        # Get response with timeout
        timer = core.Clock()
        keys = event.waitKeys(
            maxWait=self.settings['decision_duration'],
            keyList=['left', 'right', 'q'],
            timeStamped=timer
        )

        if keys and 'q' in keys[0]:
            self.win.close()
            core.quit()

        # Process response
        if not keys:
            self.send_marker(self.markers['timeout'])
            self.stim['timeout_msg'].draw()
            self.win.flip()
            core.wait(1.0)
            feedback_type = 'neutral'
            rt = None
            choice = None
            chosen_symbol = None
        else:
            self.send_marker(self.markers['choice_made'])
            key, rt = keys[0]
            choice = 0 if key == 'left' else 1
            chosen_symbol = indices[choice]

            # Determine outcome
            win_prob = self.get_win_probability(
                chosen_symbol, trial_num, n_trials, is_reversed
            )
            outcome = np.random.random() < win_prob
            feedback_type = 'win' if outcome else 'loss'

        # Wait for first R-peak after choice
        r_peak = self.r_peak_times.get()

        # Generate timing options using adaptive R-R prediction
        timing_options = self.get_timing_options(r_peak)

        # Select feedback time
        feedback_time = self.select_feedback_time(timing_mode, timing_options)

        # Store expected R-R interval for validation
        expected_r_r = self.cardiac_tracker.get_average_r_r()

        # Validate timing
        self.validate_timing(feedback_time, r_peak, timing_mode)

        # Wait until selected feedback time
        while time.time() < feedback_time:
            core.wait(0.001)

        # Show feedback
        feedback_marker = {
            'win': self.markers['win_feedback'],
            'loss': self.markers['loss_feedback'],
            'neutral': self.markers['neutral_feedback']
        }[feedback_type]

        self.send_marker(self.markers['feedback_onset'])
        self.send_marker(feedback_marker)

        self.stim['feedback'][feedback_type].draw()
        actual_feedback_time = time.time()
        self.win.flip()
        core.wait(self.settings['feedback_duration'])

        # Send trial end marker
        self.send_marker(self.markers['trial_end'])

        # Calculate timing information
        time_from_r_peak = actual_feedback_time - r_peak
        cardiac_phase = 'systole' if time_from_r_peak <= self.settings['systole_duration'] else 'diastole'

        # Create trial data
        trial_data = {
            'rt': rt,
            'choice': choice,
            'chosen_symbol': 'A' if chosen_symbol == 0 else 'B',
            'symbol_positions': 'AB' if indices[0] == 0 else 'BA',
            'feedback': feedback_type,
            'is_reversed': is_reversed,
            'timing_mode': timing_mode,
            'feedback_time': actual_feedback_time - self.experiment_start_time,
            'r_peak_time': r_peak - self.experiment_start_time,
            'cardiac_phase': cardiac_phase,
            'time_from_r_peak': time_from_r_peak,
            'expected_r_r': expected_r_r,
            'response_time': choice_time - self.experiment_start_time
        }

        # Print trial report
        self.print_trial_report(trial_data, trial_num, is_reversed)

        # Inter-trial interval
        self.stim['fixation'].draw()
        self.win.flip()
        core.wait(np.random.uniform(
            self.settings['iti_min'],
            self.settings['iti_max']
        ))

        return trial_data

    def run_experiment(self):
        """Run the complete experiment with markers"""
        # Initialize data storage
        all_data = []
        try:
            # Setup
            participant_info = self.get_participant_info()
            self.setup_experiment()

            # Initialize experiment start time
            self.experiment_start_time = time.time()

            # Send experiment start marker
            self.send_marker(self.markers['experiment_start'])

            # Setup LSL
            self.setup_lsl()
            print("Looking for R-peak stream...")
            core.wait(3.0)

            # Show instructions and run calibration
            self.show_instructions()
            self.run_calibration()

            # Create data storage
            all_data = []

            # Run blocks
            for block in range(participant_info['n_blocks']):
                self.send_marker(self.markers['block_start'])
                print(f"\nStarting Block {block + 1}...")

                n_trials = participant_info['n_trials']
                reversal_point = n_trials // 2

                for trial in range(n_trials):
                    is_reversed = trial >= reversal_point
                    if trial == reversal_point:
                        self.send_marker(self.markers['reversal'])
                        print("\nProbability Reversal!")

                    trial_data = self.run_trial(trial, n_trials, is_reversed)
                    trial_data.update({
                        'block': block,
                        'trial': trial,
                        'participant': participant_info['participant'],
                        'session': participant_info['session'],
                        'run': participant_info['run'],
                        'date_time': participant_info['date_time']
                    })
                    all_data.append(trial_data)

                self.send_marker(self.markers['block_end'])
                print(f"\nBlock {block + 1} completed.")

        finally:
            # Send experiment end marker
            self.send_marker(self.markers['experiment_end'])

            if all_data:
                # Save trial data
                df = pd.DataFrame(all_data)
                filename = f"{participant_info['participant']}-ses{participant_info['session']}-run{participant_info['run']}-{participant_info['date_time']}.csv"
                df.to_csv(self.data_path / filename, index=False)
                print(f"\nData saved as: {filename}")

                # Save timing log
                if self.timing_log:
                    log_filename = f"timing_log-{participant_info['date_time']}.csv"
                    pd.DataFrame(self.timing_log).to_csv(
                        self.data_path / log_filename, index=False
                    )
                    print(f"Timing log saved as: {log_filename}")

            self.win.close()


if __name__ == "__main__":
    task = CardiacSyncedLearningTask()
    task.run_experiment()